# Projeto 2 - Ciência dos Dados

Nome: _____

Nome: Diogo Nobre de Araujo Cintra

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [5]:
%%capture
!pip install tweepy

In [6]:
%matplotlib inline
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import matplotlib.pyplot as plt
from numpy import arange
from numpy import percentile
import numpy as np
import os
import re 

ModuleNotFoundError: No module named 'matplotlib'

___
## Autenticando no  Twitter

* Conta: ***@Ciencia dos dados***

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


Capturando os dados do twitter:

Salvando os dados em uma planilha Excel:

___
### Classificando as mensagens na coragem

#### 0 = irrelevante
#### 1 = relevante

In [4]:
TREINAMENTO = pd.read_excel('Correios.xlsx',sheet_name = 'Treinamento')
TESTE = pd.read_excel('Correios.xlsx',sheet_name = 'Teste')

In [5]:
TREINAMENTO_RELEVANTE = TREINAMENTO.loc[TREINAMENTO["Classificacao"]==1]
TREINAMENTO_IRRELEVANTE = TREINAMENTO.loc[TREINAMENTO["Classificacao"]==0]

def clean(text):
    punctuation = '[!\-.:?;#$%&*_1234567890"]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [6]:
texto_relevante = " ".join(TREINAMENTO_RELEVANTE['Treinamento'])
texto_relevante_1 = clean(texto_relevante)
texto_relevante_2 = texto_relevante_1.split()
texto_relevante_3 = []

texto_irrelevante = " ".join(TREINAMENTO_IRRELEVANTE['Treinamento'])
texto_irrelevante_1 = clean(texto_irrelevante)
texto_irrelevante_2 = texto_irrelevante_1.split()
texto_irrelevante_3 = []

treinamento = " ".join(TREINAMENTO['Treinamento'])
treinamento_1 = clean(treinamento)
treinamento_2 = treinamento_1.split()
treinamento_3 = []

for e in range(len(texto_relevante_2)-1):
    if texto_relevante_2[e] != 'rt' and texto_relevante_2[e][0] != '@':     
        texto_relevante_3.append(texto_relevante_2[e])    
        
for e in range(len(texto_irrelevante_2)-1):
    if texto_irrelevante_2[e] != 'rt' and texto_irrelevante_2[e][0] != '@':     
        texto_irrelevante_3.append(texto_irrelevante_2[e]) 
        
for e in range(len(treinamento_2)-1):
    if treinamento_2[e] != 'rt' and treinamento_2[e][0] != '@':     
        treinamento_3.append(treinamento_2[e])
        
texto_relevante_4 = pd.Series(texto_relevante_3)
texto_irrelevante_4 = pd.Series(texto_irrelevante_3)
treinamento_4 = pd.Series(treinamento_3)

#### Probabilidade de uma palavra ser relevante

In [12]:
#total_amostral = treinamento_4.value_counts()
#total_relevante = texto_relevante_4.value_counts()

TREINAMENTO.Classificacao.value_counts()
prev = 120/300 #probabilidade de ser relevante geral

total_amostral = len(treinamento_4)-1
total_relevante = len(texto_relevante_4)
prob_relevante = total_relevante/total_amostral
prob_relevante

0.4027777777777778

#### Probabilidade de uma palavra ser irrelevante

In [8]:
#total_amostral = treinamento_4.value_counts()
#total_relevante = texto_relevante_4.value_counts()

TREINAMENTO.Classificacao.value_counts()
prev = 180/300 #probabilidade de ser relevante geral

total_amostral = len(treinamento_4)-1
total_irrelevante = len(texto_irrelevante_4)
prob_irrelevante = total_irrelevante/total_amostral
prob_irrelevante

0.5972222222222222

#probabilidade de ser relevante
TREINAMENTO.Classificacao.value_counts()
prev = 120/300

In [9]:
#probabilidade de não ser relevante
pnorev = 180/300

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [12]:
#contador das palavras
def conta_palavra(texto, dic):
    texto_sem_pontuacao = clean(texto)
    palavras_do_texto = texto_sem_pontuacao.split()
    palavras=dic
    for p in palavras_do_texto:
        if p in palavras:
            palavras[p]+=1
        else:
            palavras[p]=1
    return palavras

In [13]:
#cria database setarada de relevantes e irrelevantes
data_frame_rlevantes = TREINAMENTO.loc[TREINAMENTO["Classificacao"]==1]
serie_relevantes=pd.Series(data_frame_rlevantes["Treinamento"])
data_frame_nao_rlevantes = TREINAMENTO.loc[TREINAMENTO["Classificacao"]==0]
serie_nao_relevantes=pd.Series(data_frame_nao_rlevantes["Treinamento"])
#conta a incidencia de cada palavra
contagem_relevantes={}
for T in serie_relevantes:
    contagem_relevantes=conta_palavra(T,contagem_relevantes)
contagem_nao_relevantes={}
for T in serie_nao_relevantes:
    serie_nao_relevantes=conta_palavra(T,contagem_nao_relevantes)

#associa um valor de probabilidade
probrev={}

#formato {palavra:valor}
probnorev={}
#formato {palavra:valor}
serie_nao_relevantes

{'rt': 69,
 '@arthurmoledoval': 17,
 'o': 104,
 'governo': 23,
 'recomendou': 15,
 'ao': 22,
 'bndes': 16,
 'que': 103,
 'os': 66,
 'estudos': 15,
 'sobre': 22,
 'uma': 44,
 'eventual': 15,
 'privatização': 20,
 'dos': 58,
 'correios': 135,
 'fiquem': 15,
 'prontos': 15,
 'em': 43,
 'até': 16,
 'mese…': 15,
 '@casonatogabriel': 11,
 'a': 84,
 'amazon,': 14,
 'empresa': 16,
 'na': 34,
 'qual': 12,
 'queremos': 11,
 'transformar': 22,
 'correios,': 26,
 'lança': 11,
 'hoje': 15,
 'no': 37,
 'brasil': 15,
 'programa': 11,
 'prime,': 11,
 'dá': 11,
 'aos': 15,
 'seus': 12,
 'ass…': 11,
 '@willianlofy': 1,
 'usam': 2,
 'quem': 3,
 'quer,': 1,
 'como': 15,
 '“todos”': 1,
 'sabem,': 1,
 'monopólio': 4,
 'é': 38,
 'somente': 1,
 'daquelas': 1,
 'cartinhas': 1,
 'ninguém': 2,
 'mais': 16,
 'usa': 1,
 'pessoa': 3,
 'vai': 14,
 'dirigir': 1,
 'de': 120,
 'madrugada': 1,
 'achando': 1,
 'braian': 1,
 'ai': 3,
 'dps': 1,
 'fica': 2,
 'um': 17,
 'mês': 2,
 'indo': 2,
 'caixinha': 2,
 'todo': 6,
 'di

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [85]:
smoth = 1/1000000 #é uma estimativa e deve mudar
# defini probabilidade de ser relevante:
def relevante(tweet):
    t = clean(tweet)
    prob = 1
    for p in t:
        if p in probrev:
            prob = prob*(probrev(p)+smoth)
        else:
            prob = prob*(smoth)
    prob = prev
# defini probabilidade de ser irrelevante:
def relevante(tweet):
    t = clean(tweet)
    prob = 1
    for p in t:
        if p in probnorev:
            prob = prob*(probnorev(p)+smoth)
        else:
            prob = prob*(smoth)
    prob = pnorev

In [86]:
def compara (tweet):
    r = relevante(tweet)
    nor = irrelevante(tweet)
    return r>nor

In [87]:
arquivo = 
rev = 0
norev = 0
for txt in arquivo:
    if compara (txt):
        rev +=1
    else:
        norev+=1
print ("a probabilidade de ser relevante é {}".format(rev/arquivo.len()))
print ("a probabilidade de não ser relevante é {}".format(rev/arquivo.len()))
if rev/arquivo.len() < 0.9*prev:
    print ("poucos relevantes")
elif rev/arquivo.len() > 1.1*prev:
    print ("muitos relevantes")
else:
    print ("ok: {} veze o esperado".format (rev/arquivo.len()/prev))


SyntaxError: invalid syntax (<ipython-input-87-590e40ef14d4>, line 1)

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**